# 1.Import Dependencies

In [ ]:
import os
from cmath import pi

import gymnasium
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [ ]:
environment_name = 'CartPole-v1'
env = gymnasium.make(environment_name, render_mode="human")

In [ ]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward

    print(f'Episode:{episode}, Score:{score}')
env.close() 

# 3.Understanding the Environment

In [ ]:
env.action_space

In [ ]:
env.action_space.sample()

In [ ]:
env.observation_space.sample()

# 4. Train Model

In [ ]:
log_path = os.path.join('Training', 'Logs')


In [ ]:
import torch

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    print("MPS is not available, using CPU.")



In [ ]:
env = gymnasium.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, device=device)

In [ ]:
model.learn(total_timesteps=20000)

# 5. Save Model & Load Model

In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model_CartPole')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load(PPO_path, env=env)

# 6. Evaluate

In [ ]:
# Step 1: Recreate the environment
env = gymnasium.make("CartPole-v1", render_mode="human")

# Step 2: Define the path to the saved model
PPO_path = os.path.join("Training", "Saved Models", "PPO_model_CartPole")

# Step 3: Load the saved model
model = PPO.load(PPO_path, env=env)

# Step 4: Custom evaluation function
def evaluate_policy_with_rendering(model, env, n_eval_episodes=10, render=True):
    """
    Evaluate the policy of a loaded model with optional rendering.
    """
    episode_rewards = []

    for episode in range(n_eval_episodes):
        reset_output = env.reset()
        state = reset_output[0] if isinstance(reset_output, tuple) else reset_output
        done = False
        total_reward = 0

        while not done:
            if render:
                env.render()  # Render each frame

            # Get the action from the model
            action, _states = model.predict(state, deterministic=True)
            
            # Adjust for environments returning 4 or 5 values
            step_output = env.step(action)
            if len(step_output) == 5:
                state, reward, terminated, truncated, info = step_output
            else:
                state, reward, terminated, truncated = step_output

            # Convert reward to scalar to avoid warnings
            total_reward += reward.item() if hasattr(reward, 'item') else float(reward)

            # Combine termination flags
            done = terminated or truncated

        episode_rewards.append(total_reward)
        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    # Calculate mean and standard deviation of rewards
    mean_reward = sum(episode_rewards) / n_eval_episodes
    std_reward = (sum([(x - mean_reward) ** 2 for x in episode_rewards]) / n_eval_episodes) ** 0.5

    return mean_reward, std_reward

# Step 5: Evaluate the loaded model
mean_reward, std_reward = evaluate_policy_with_rendering(model, env, n_eval_episodes=10, render=True)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

# Close the environment
env.close()

# 7. Test Model

In [ ]:
import os
import gymnasium as gym
from stable_baselines3 import PPO

# Create the environment
env = gym.make("CartPole-v1", render_mode="human")

# Define the path to the saved model
PPO_path = os.path.join("Training", "Saved Models", "PPO_model_CartPole")

# Load the saved model
model = PPO.load(PPO_path, env=env)

# Number of episodes
episodes = 10

for episode in range(1, episodes + 1):
    # Extract the observation from the reset output
    obs, _ = env.reset()
    done = False
    score = 0

    while not done:
        action, _states = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward

    print(f'Episode: {episode}, Score: {score}')

# Close the environment
env.close()

# 8. View Logs in Tensorboard

In [ ]:
import os
log_path = os.path.join('Training', 'Logs')
training_log_path = os.path.join(log_path, 'PPO_2')


In [ ]:
!tensorboard --logdir={training_log_path}

# 9. Adding a callback to the training stage

In [ ]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.monitor import Monitor

# Create the environment
env = gym.make("CartPole-v1", render_mode="human")

# Wrap the training environment with Monitor for proper episode length and reward tracking
train_env = Monitor(env)

# The path to save logs and models
log_path = os.path.join("Training", "Logs")
best_model_save_path = os.path.join("Training", "Saved_Models")

# Create the evaluation environment and wrap it with Monitor
eval_env = gym.make("CartPole-v1")
eval_env = Monitor(eval_env)

# Create the callback
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(eval_env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=best_model_save_path,
                             verbose=1)

# Create the PPO model
model = PPO('MlpPolicy', train_env, verbose=1, tensorboard_log=log_path)

# Train the model
model.learn(total_timesteps=20000, callback=eval_callback)

# Save the trained model
model_save_path = os.path.join("Training", "Saved_Models", "ppo_cartpole")
model.save(model_save_path)

print("Training complete! Model saved at:", model_save_path)

# Test the trained model
test_env = gym.make("CartPole-v1", render_mode="human")
obs = test_env.reset()
done = False

print("Testing the trained model...")
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, _ = test_env.step(action)
    if done:
        print("Episode finished!")
        break

# Close the environment
test_env.close()

# 10. Change policy

In [ ]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3.common.monitor import Monitor
import torch

# Check if MPS is available
device = torch.device("mps" if torch.has_mps else "cpu")

# Create the environment
env = gym.make("CartPole-v1", render_mode="human")

# Wrap the training environment with Monitor for proper episode length and reward tracking
train_env = Monitor(env)

# The path to save logs and models
log_path = os.path.join("Training", "Logs")
best_model_save_path = os.path.join("Training", "Saved_Models")

# Create the evaluation environment and wrap it with Monitor
eval_env = gym.make("CartPole-v1")
eval_env = Monitor(eval_env)

# Create the callback
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(eval_env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=best_model_save_path,
                             verbose=1)

# Define the new policy architecture
new_arch = dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])

# Create the PPO model with the new policy architecture
model = PPO('MlpPolicy', train_env, policy_kwargs={"net_arch": new_arch}, verbose=1, tensorboard_log=log_path, device=device)

# Train the model
model.learn(total_timesteps=20000, callback=eval_callback)

# Save the trained model
model_save_path = os.path.join("Training", "Saved_Models", "ppo_cartpole")
model.save(model_save_path)

print("Training complete! Model saved at:", model_save_path)

# Test the trained model
test_env = gym.make("CartPole-v1", render_mode="human")
episodes = 10

for episode in range(1, episodes + 1):
    obs, _ = env.reset()
    done = False
    score = 0

    while not done:
        action, _states = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward

    print(f'Episode: {episode}, Score: {score}')

# Close the environment
env.close()

# 11. Utilising an Alternate Algorithim

In [1]:
import os
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Create the environment
env = gym.make("CartPole-v1")
env = Monitor(env)
env = DummyVecEnv([lambda: env])
env = VecNormalize(env, norm_reward=True)

# Log and model paths
log_path = os.path.join("Training", "Logs")
model_save_path = os.path.join("Training", "Saved_Models", "dqn_cartpole")

# Create the DQN model
model = DQN(
    'MlpPolicy',
    env,
    verbose=1,
    tensorboard_log=log_path,
    exploration_fraction=0.3,  # Slower exploration decay
    exploration_final_eps=0.05,  # Ensure some exploration remains
    buffer_size=100000,  # Larger replay buffer
)

# Train the model
model.learn(total_timesteps=100000)

# Save the trained model
model.save(model_save_path)
print("Training complete! Model saved at:", model_save_path)

# Test the trained model
test_env = gym.make("CartPole-v1", render_mode="human")
episodes = 10

for episode in range(1, episodes + 1):
    obs, _ = test_env.reset()
    done = False
    score = 0

    while not done:
        action, _states = model.predict(obs)
        obs, reward, terminated, truncated, info = test_env.step(action)
        done = terminated or truncated
        score += reward

    print(f'Episode: {episode}, Score: {score}')

# Close the environment
env.close()

Using cpu device
Logging to Training/Logs/DQN_15
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.2     |
|    ep_rew_mean      | 22.2     |
|    exploration_rate | 0.997    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 9237     |
|    time_elapsed     | 0        |
|    total_timesteps  | 89       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 22.9     |
|    ep_rew_mean      | 22.9     |
|    exploration_rate | 0.994    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 4322     |
|    time_elapsed     | 0        |
|    total_timesteps  | 183      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00639  |
|    n_updates        | 20       |
----------------------------------
----------------------------------
| roll

2024-12-20 15:38:21.458 Python[7524:327373] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-20 15:38:21.458 Python[7524:327373] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Episode: 1, Score: 13.0
Episode: 2, Score: 14.0
Episode: 3, Score: 13.0
Episode: 4, Score: 10.0
Episode: 5, Score: 11.0
Episode: 6, Score: 11.0
Episode: 7, Score: 12.0
Episode: 8, Score: 12.0
Episode: 9, Score: 14.0
Episode: 10, Score: 11.0
